## ANLY 5336 Analytics Project: COVID-19 Forecasting
Author: John Courtright

This project works through data from the New York Time's COVID-19 datasets. Link: https://github.com/nytimes/covid-19-data

This notebook contains script to find a suitable county to focus on for the project. The group is looking for a county that has suitable, complete data in the various CSV files from the repository: We want a county that has complete data from 2020-2023 in:

    (1) rolling-averages
    (2) mask-use
    (3) a complete 3-year pool of data

In [2]:
# Load data sets
import pandas as pd

df_2020 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2020.csv')
df_2021 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2021.csv')
df_2022 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2022.csv')
df_2023 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2023.csv')
# df_counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties.csv')
df_mask = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/mask-use/mask-use-by-county.csv')

In the yearly data sets, the fields have the following definitions:

    geoid: A unique geographic identifier for each place. For counties and states, the final five digits are the same as the FIPS code when possible. In instances where we have assigned a non-standard identifier, the geoid will end in 99[0-9].
    cases: The number of new cases of Covid-19 reported that day, including both confirmed and probable.
    cases_avg: The average number of new cases reported over the most recent seven days of data.
    cases_avg_per_100k: The cases_avg per 100,000 people.
    deaths: The total number of new deaths from Covid-19 reported that day, including both confirmed and probable.
    deaths_avg: The daily average number of new deaths reported over the most recent period. County-level deaths are averaged over 30 days. State, territory and national deaths are averaged over 7 days.
    deaths_avg_per_100k: The deaths_avg per 100,000 people.

In the mask survey, the fields have the following definitions:

    COUNTYFP: The county FIPS code.
    NEVER: The estimated share of people in this county who would say never in response to the question “How often do you wear a mask in public when you expect to be within six feet of another person?”
    RARELY: The estimated share of people in this county who would say rarely
    SOMETIMES: The estimated share of people in this county who would say sometimes
    FREQUENTLY: The estimated share of people in this county who would say frequently
    ALWAYS: The estimated share of people in this county who would say always

In [ ]:
# Stack yearly data frames vertically and find counties with a complete 3-year weekly series (2020-01-01 through 2022-12-31).
# This cell relies on df_2020, df_2021, df_2022, df_2023 already defined in the notebook.

# concatenate
df_all = pd.concat([df_2020, df_2021, df_2022, df_2023], ignore_index=True)

# ensure datetime
df_all['date'] = pd.to_datetime(df_all['date'])

# determine county identifier column (try common options)
if 'geoid' in df_all.columns:
    id_col = 'geoid'
elif 'fips' in df_all.columns:
    id_col = 'fips'
elif 'COUNTYFP' in df_all.columns:
    id_col = 'COUNTYFP'
else:
    # fallback to any column that looks like a county identifier
    possible = [c for c in df_all.columns if 'county' in c.lower() or 'fips' in c.lower() or 'geo' in c.lower()]
    id_col = possible[0] if possible else 'county'

# numeric columns we'll check for completeness (use those that actually exist)
candidate_cols = ['cases_avg_per_100k', 'cases_avg', 'deaths_avg', 'deaths_avg_per_100k', 'cases', 'deaths']
check_cols = [c for c in candidate_cols if c in df_all.columns]
if not check_cols:
    raise RuntimeError(f"No expected numeric columns found in df_all. Available columns: {df_all.columns.tolist()}")

# define 3-year span (2020-03-13 through 2023-3-13) and weekly index
start, end = '2020-03-13', '2023-3-13'
weekly_index = pd.date_range(start=start, end=end, freq='W')

# group, resample weekly and test for any missing values across the chosen check columns
good_counties = []
for gid, grp in df_all.groupby(id_col):
    g = grp.sort_values('date').set_index('date')
    # resample weekly and aggregate by mean (appropriate for averages); produces NaN for empty weeks
    weekly = g[check_cols].resample('W').mean()
    # reindex to the full target weekly index to detect any missing weeks
    weekly = weekly.reindex(weekly_index)
    if not weekly.isnull().any().any():
        good_counties.append(gid)

# results dataframe
good_counties_df = pd.DataFrame({id_col: good_counties})
print(f"Counties with complete weekly data from {start} to {end}: {len(good_counties_df)}")
good_counties_df.head(50)

Counties with complete weekly data from 2020-03-13 to 2023-3-13: 424


,geoid
0,USA-01003
1,USA-01051
2,USA-01073
3,USA-01081
4,USA-01083
5,USA-01101
6,USA-01117
7,USA-01125
8,USA-02020
9,USA-04009


In [4]:
# Build a complete weekly DataFrame for the counties listed in good_counties_df.
# Relies on variables already defined in the notebook: df_all, id_col, check_cols, weekly_index, good_counties_df

# Detect a county name column if present
name_cols = [c for c in df_all.columns if 'county' in c.lower() or c.lower() == 'name']
name_col = name_cols[0] if name_cols else None

# Detect a FIPS column (or derive from geoid)
if 'fips' in df_all.columns:
    fips_col = 'fips'
elif 'FIPS' in df_all.columns:
    fips_col = 'FIPS'
elif 'COUNTYFP' in df_all.columns:
    fips_col = 'COUNTYFP'
else:
    fips_col = None

def _format_fips(val, fallback_gid=None):
    # normalize to 5-digit string where possible
    if pd.isna(val) and fallback_gid is not None:
        val = fallback_gid
    try:
        # handle floats and ints
        return str(int(float(val))).zfill(5)
    except Exception:
        s = str(val)
        # if geoid-like, take last 5 chars
        if len(s) >= 5:
            return s[-5:].zfill(5)
        return s.zfill(5)

rows = []
for gid in good_counties_df[id_col].tolist():
    grp = df_all[df_all[id_col] == gid].sort_values('date').set_index('date')
    # resample weekly using the already-determined numeric columns
    weekly = grp[check_cols].resample('W').mean()
    weekly = weekly.reindex(weekly_index)  # ensure full weekly span
    
    # determine a representative county name and fips for this gid
    county_name = None
    if name_col and name_col in grp.columns:
        # take the most common (mode) value if available
        try:
            county_name = grp[name_col].mode().iloc[0]
        except Exception:
            county_name = grp[name_col].iloc[0]
    # determine fips: prefer explicit fips column; else derive from gid when possible
    if fips_col and fips_col in grp.columns:
        raw_fips = grp[fips_col].iloc[0]
    elif id_col == 'geoid':
        raw_fips = str(gid)[-5:]
    else:
        raw_fips = None
    fips = _format_fips(raw_fips, fallback_gid=gid)
    name_val = county_name if county_name is not None else str(gid)
    weekly = weekly.copy()
    weekly[id_col] = gid
    weekly['county'] = name_val
    weekly['fips'] = fips
    weekly['county_fips_name'] = f"{name_val} ({fips})"
    rows.append(weekly)

# concatenate all counties into one DataFrame and tidy up
df_weekly_counties = pd.concat(rows, axis=0).reset_index().rename(columns={'index': 'date'})
# reorder columns: date, fips, county, county_fips_name, id_col, then numeric columns
numeric_cols = [c for c in check_cols if c in df_weekly_counties.columns]
ordered_cols = ['date', 'fips', 'county', 'county_fips_name', id_col] + numeric_cols
# keep only columns that exist
ordered_cols = [c for c in ordered_cols if c in df_weekly_counties.columns]
df_weekly_counties = df_weekly_counties[ordered_cols]

print(f"Built weekly series for {df_weekly_counties['fips'].nunique()} counties, rows: {len(df_weekly_counties)}")
df_weekly_counties.head()

Built weekly series for 424 counties, rows: 66568


,date,fips,county,county_fips_name,geoid,cases_avg_per_100k,cases_avg,deaths_avg,deaths_avg_per_100k,cases,deaths
0,2020-03-15,01003,Baldwin,Baldwin (01003),USA-01003,0.060000,0.140000,0.000000,0.000000,0.500000,0.000000
1,2020-03-22,01003,Baldwin,Baldwin (01003),USA-01003,0.090000,0.204286,0.000000,0.000000,0.142857,0.000000
2,2020-03-29,01003,Baldwin,Baldwin (01003),USA-01003,0.251429,0.560000,0.000000,0.000000,1.857143,0.000000
3,2020-04-05,01003,Baldwin,Baldwin (01003),USA-01003,1.141429,2.545714,0.021429,0.007143,2.714286,0.142857
4,2020-04-12,01003,Baldwin,Baldwin (01003),USA-01003,1.930000,4.308571,0.030000,0.010000,5.285714,0.000000


In [ ]:
# Print top 10 counties by cases_avg_per_100k and by deaths_avg_per_100k
# Relies on df_weekly_counties already defined in the notebook.

def _print_top10(metric):
    if metric not in df_weekly_counties.columns:
        print(f"Column '{metric}' not found in df_weekly_counties.")
        return
    top = (
        df_weekly_counties
        .groupby('county_fips_name', as_index=False)[metric]
        .median()
        .sort_values(metric, ascending=False)
        .head(10)
    )
    print(f"\nTop 10 counties by {metric} (mean weekly value):")
    print(top.to_string(index=False))

_print_top10('cases_avg_per_100k')
_print_top10('deaths_avg_per_100k')


Top 10 counties by cases_avg_per_100k (mean weekly value):
  county_fips_name  cases_avg_per_100k
Miami-Dade (12086)           32.001429
     Clark (21049)           27.854286
Montgomery (21173)           27.542857
    Nassau (36059)           25.621429
   Suffolk (36103)           25.391429
     Ocean (34029)           25.338571
    Orange (36071)           25.131429
  Harrison (21097)           23.772857
  Monmouth (34025)           23.732857
  Sullivan (47163)           23.482857

Top 10 counties by deaths_avg_per_100k (mean weekly value):
 county_fips_name  deaths_avg_per_100k
   Citrus (12017)             0.534286
 Columbia (53013)             0.510000
  Leflore (28083)             0.440000
  Garland (05051)             0.401429
      Bay (26017)             0.390000
    Caddo (22017)             0.388571
Charlotte (12015)             0.388571
Dougherty (13095)             0.377143
   Copiah (28029)             0.362857
  Volusia (12127)             0.360000


In [11]:
# Build a weekly DataFrame for Texas counties over the same 3-year weekly span.
# Relies on df_all, id_col, check_cols, weekly_index, _format_fips, name_col, fips_col already defined.

# detect state column
state_cols = [c for c in df_all.columns if c.lower() == 'state' or 'state' in c.lower()]
if not state_cols:
    raise RuntimeError("No state column found in df_all.")
state_col = state_cols[0]

# filter to Texas (case-insensitive)
tx_df = df_all[df_all[state_col].astype(str).str.strip().str.lower() == 'texas'].copy()
if tx_df.empty:
    raise RuntimeError("No rows for Texas found in df_all.")

rows = []
for gid, grp in tx_df.groupby(id_col):
    g = grp.sort_values('date').set_index('date')
    weekly = g[check_cols].resample('W').mean()
    weekly = weekly.reindex(weekly_index)
    # county name
    county_name = None
    if name_col and name_col in g.columns:
        try:
            county_name = g[name_col].mode().iloc[0]
        except Exception:
            county_name = g[name_col].iloc[0]
    # fips
    if fips_col and fips_col in g.columns:
        raw_fips = g[fips_col].iloc[0]
    elif id_col == 'geoid':
        raw_fips = str(gid)[-5:]
    else:
        raw_fips = None
    fips = _format_fips(raw_fips, fallback_gid=gid)
    name_val = county_name if county_name is not None else str(gid)
    weekly = weekly.copy()
    weekly[id_col] = gid
    weekly['county'] = name_val
    weekly['fips'] = fips
    weekly['county_fips_name'] = f"{name_val} ({fips})"
    rows.append(weekly)

df_weekly_texas = pd.concat(rows, axis=0).reset_index().rename(columns={'index': 'date'})

# reorder columns: date, fips, county, county_fips_name, id_col, numeric cols
numeric_cols = [c for c in check_cols if c in df_weekly_texas.columns]
ordered_cols = ['date', 'fips', 'county', 'county_fips_name', id_col] + numeric_cols
ordered_cols = [c for c in ordered_cols if c in df_weekly_texas.columns]
df_weekly_texas = df_weekly_texas[ordered_cols]

print(f"Built weekly series for Texas: {df_weekly_texas['fips'].nunique()} counties, rows: {len(df_weekly_texas)}")
df_weekly_texas.head(50)

Built weekly series for Texas: 255 counties, rows: 40035


,date,fips,county,county_fips_name,geoid,cases_avg_per_100k,cases_avg,deaths_avg,deaths_avg_per_100k,cases,deaths
0,2020-03-15,48001,Anderson,Anderson (48001),USA-48001,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-22,48001,Anderson,Anderson (48001),USA-48001,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-29,48001,Anderson,Anderson (48001),USA-48001,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-05,48001,Anderson,Anderson (48001),USA-48001,0.250000,0.140000,0.000000,0.000000,0.200000,0.000000
4,2020-04-12,48001,Anderson,Anderson (48001),USA-48001,0.301429,0.172857,0.000000,0.000000,0.285714,0.000000
5,2020-04-19,48001,Anderson,Anderson (48001),USA-48001,0.722857,0.417143,0.000000,0.000000,0.714286,0.000000
6,2020-04-26,48001,Anderson,Anderson (48001),USA-48001,1.972857,1.141429,0.000000,0.000000,1.428571,0.000000
7,2020-05-03,48001,Anderson,Anderson (48001),USA-48001,3.207143,1.854286,0.000000,0.000000,2.000000,0.000000
8,2020-05-10,48001,Anderson,Anderson (48001),USA-48001,2.901429,1.675714,0.000000,0.000000,1.285714,0.000000
9,2020-05-17,48001,Anderson,Anderson (48001),USA-48001,3.100000,1.792857,0.000000,0.000000,1.857143,0.000000


In [13]:
df_weekly_texas['county_fips_name'].value_counts()

county_fips_name
Anderson (48001)     157
Newton (48351)       157
Medina (48325)       157
Menard (48327)       157
Midland (48329)      157
                    ... 
Grayson (48181)      157
Gregg (48183)        157
Grimes (48185)       157
Guadalupe (48187)    157
Unknown (48999)      157
Name: count, Length: 255, dtype: int64

In [25]:
def _print_top_n_texas(candidate_metrics, label, n=10):
    # pick the first existing metric name from candidates
    metric = next((m for m in candidate_metrics if m in df_weekly_texas.columns), None)
    if metric is None:
        print(f"No metric found for {label}. Tried: {candidate_metrics}")
        return
    top = (
        df_weekly_texas
        .groupby('county_fips_name', as_index=False)[metric]
        .mean()
        .dropna(subset=[metric])
        .sort_values(metric, ascending=False)
        .head(n)
    )
    print(f"\nTop {n} Texas counties by {label} (mean weekly value of '{metric}'):")
    print(top.to_string(index=False))

_print_top_n_texas(['cases_avg_per_100k', 'cases_avg'], 'cases per 100k')
_print_top_n_texas(['deaths_avg_per_100k', 'deaths_avg'], 'deaths per 100k')


Top 10 Texas counties by cases per 100k (mean weekly value of 'cases_avg_per_100k'):
 county_fips_name  cases_avg_per_100k
   Loving (48301)          292.285159
 Jim Hogg (48247)           68.585828
   Dimmit (48127)           56.828161
   Concho (48095)           55.639384
   Scurry (48415)           46.842582
 Crockett (48105)           45.955704
Childress (48075)           42.830414
     Hall (48191)           41.771961
    Irion (48235)           41.761340
Tom Green (48451)           40.142431

Top 10 Texas counties by deaths per 100k (mean weekly value of 'deaths_avg_per_100k'):
    county_fips_name  deaths_avg_per_100k
    McMullen (48311)             2.015797
      Baylor (48023)             1.439507
       Foard (48155)             1.272816
      Motley (48345)             1.219821
      Dallam (48111)             1.205285
        Hall (48191)             1.175743
       Floyd (48153)             1.173931
    Sterling (48431)             1.132709
        Lamb (48279)          

In [20]:
colleges_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/colleges/colleges.csv')
colleges_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948 entries, 0 to 1947
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1948 non-null   object 
 1   state       1948 non-null   object 
 2   county      1946 non-null   object 
 3   city        1948 non-null   object 
 4   ipeds_id    1948 non-null   object 
 5   college     1948 non-null   object 
 6   cases       1948 non-null   int64  
 7   cases_2021  1611 non-null   float64
 8   notes       36 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 137.1+ KB


In [29]:
# find colleges in Essex County, New Jersey using existing colleges_df
mask = (
    (colleges_df['state'].astype(str).str.strip().str.lower() == 'texas') &
    (colleges_df['county'].astype(str).str.strip().str.lower() == 'travis')
)
essex_colleges = colleges_df[mask].copy()

print(f"Rows found: {len(essex_colleges)}, Unique colleges: {essex_colleges['college'].nunique()}")
if essex_colleges.empty:
    print("No colleges found for Essex County, NJ.")
else:
    cols_to_show = [c for c in ['college', 'city', 'ipeds_id', 'cases', 'cases_2021', 'notes'] if c in essex_colleges.columns]
    print(essex_colleges[cols_to_show].drop_duplicates().reset_index(drop=True).to_string(index=False))

Rows found: 4, Unique colleges: 4
                          college   city ipeds_id  cases  cases_2021                                                          notes
Austin Community College District Austin   222992     51        22.0                                                            NaN
       Concordia University Texas Austin   224004     83        76.0                                                            NaN
        Saint Edward's University Austin   227845     47        27.0 College did not clarify whether data includes duplicate cases.
    University of Texas at Austin Austin   228778   3989      1606.0                                                            NaN
